# 시카고 샌드위치 맛집 페이지 분석
- URL : https://www.chicagomag.com/chicago-magazine/november-2012/best-sandwiches-chicago/

---
## 1. BeautifulSoup로 웹 페이지 html 파싱하기
- 추후 모든 순위 메뉴의 상세 정보 페이지에 접근하여 가격과 가게 주소 정보를 가져오기 위해 url_base와 url_sub로 url값을 나누어주었다.

In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as bs

url_base = "https://www.chicagomag.com/"
url_sub = "chicago-magazine/november-2012/best-sandwiches-chicago/"
url = url_base + url_sub

req = Request(url, headers={'user-agent':'Chrome'})
res = urlopen(req)

res.status

200

- BeautifulSoup를 통해 html 태그를 파싱

In [2]:
soup = bs(res, "html.parser")

print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible">
   <link href="https://gmpg.org/xfn/11" rel="profile"/>
   <script src="https://cmp.osano.com/16A1AnRt2Fn8i1unj/f15ebf08-7008-40fe-9af3-db96dc3e8266/osano.js">
   </script>
   <title>
    The 50 Best Sandwiches in Chicago – Chicago Magazine
   </title>
   <style type="text/css">
    .heateor_sss_button_instagram span.heateor_sss_svg,a.heateor_sss_instagram span.heateor_sss_svg{background:radial-gradient(circle at 30% 107%,#fdf497 0,#fdf497 5%,#fd5949 45%,#d6249f 60%,#285aeb 90%)}
						div.heateor_sss_horizontal_sharing a.heateor_sss_button_instagram span{background:#000!important;}div.heateor_sss_standard_follow_icons_container a.heateor_sss_button_instagram span{background:#000;}
										.heateor_sss_horizontal_sharing .heateor_sss_svg,.heateor_sss_standard_follow_icons_container .heateor_sss_svg{
							background-color: #000!important;
				background: #000

---
## 2. 1위 메뉴 정보 호출
- 각 순위별 메뉴 정보는 각각 div 태그로 이루어져있다.
- div 태그의 클래스는 sammy이다.

- find_all 메서드

In [3]:
len(soup.find_all("div", "sammy"))

50

- select 메서드

In [4]:
len(soup.select(".sammy"))

50

In [5]:
soup.select(".sammy")

[<div class="sammy" style="position: relative;">
 <div class="sammyRank">1</div>
 <div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
 Old Oak Tap<br/>
 <em>Read more</em> </a></div>
 </div>,
 <div class="sammy" style="position: relative;">
 <div class="sammyRank">2</div>
 <div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Au-Cheval-Fried-Bologna/"><b>Fried Bologna</b><br/>
 Au Cheval<br/>
 <em>Read more</em> </a></div>
 </div>,
 <div class="sammy" style="position: relative;">
 <div class="sammyRank">3</div>
 <div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Xoco-Woodland-Mushroom/"><b>Woodland Mushroom</b><br/>
 Xoco<br/>
 <em>Read more</em> </a></div>
 </div>,
 <div class="sammy" style="position: relative;">
 <div class="sammyRank">4</div>
 <div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-i

- 1위 메뉴를 확인해보자

In [6]:
soup.select(".sammy")[0]

<div class="sammy" style="position: relative;">
<div class="sammyRank">1</div>
<div class="sammyListing"><a href="/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/"><b>BLT</b><br/>
Old Oak Tap<br/>
<em>Read more</em> </a></div>
</div>

- 1위 메뉴의 순위, 메뉴 이름 문자열을 가져와보자.

In [7]:
tmp_one = soup.select(".sammy")[0]

# print(tmp_one.select_one(".sammyRank").get_text())
tmp_one.select_one(".sammyListing").get_text()

'BLT\nOld Oak Tap\nRead more '

- 정규표현식을 사용하여 위 결과값에서 원하는 문자열만 가져와보자.

In [11]:
import re

tmp_string = tmp_one.select_one(".sammyListing").get_text()
re.split(("\n|\r\n"), tmp_string)

['BLT', 'Old Oak Tap', 'Read more ']

- 1위 메뉴에 대한 상세 정보를 주는 페이지 주소 링크를 가져와보자.

In [9]:
tmp_one.select_one("a").get('href')

'/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

- 1위 메뉴에 대한 필요한 정보를 정리

In [12]:
# 순위
print(tmp_one.select_one(".sammyRank").get_text())
# 메뉴 이름
print(re.split(("\n|\r\n"), tmp_string)[0])
# 가게 이름
print(re.split(("\n|\r\n"), tmp_string)[1])
# 상세정보 링크
print(tmp_one.select_one("a").get("href"))

1
BLT
Old Oak Tap
/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/


---
## 3. 전체 순위 메뉴 정보 호출하기
- 위에서 1위 메뉴에 대한 필요한 정보를 가져왔다.
- 반복문을 통해 전체 순위 메뉴에 대한 정보를 가져와보자.

In [19]:
from tqdm import tqdm
from urllib.parse import urljoin

url_base = "https://www.chicagomag.com/"

# 각 리스트는 데이터프레임의 컬럼이 될 예정
rank = []   # 50개 순위
main_menu = []  # 50개 메뉴 이름
cafe_name = []  # 50개 가게 이름
url_add = []    # 50개 메뉴에 대한 상세 정보 페이지

list_soup = soup.select(".sammy")

for item in tqdm(list_soup) :
    rank.append(item.select_one(".sammyRank").get_text())

    tmp_string = item.select_one(".sammyListing").get_text()
    menu = re.split(("\n|\r\n"), tmp_string)[0]
    cafe = re.split(("\n|\r\n"), tmp_string)[1]

    main_menu.append(menu)
    cafe_name.append(cafe)

    url_add.append(urljoin(url_base, item.select_one("a").get("href")))

100%|██████████| 50/50 [00:00<00:00, 6329.69it/s]


In [20]:
len(rank), len(main_menu), len(cafe_name), len(url_add)

(50, 50, 50, 50)

---
## 4. 데이터프레임 생성
- rank, main_menu, cafe_name, url_add 리스트들로 데이터프레임을 생성해보자.

In [21]:
import pandas as pd

data = {
    "Rank" : rank,
    "Menu" : main_menu,
    "Cafe" : cafe_name,
    "URL" : url_add
}

df = pd.DataFrame(data)

df.head()

,Rank,Menu,Cafe,URL
0,1,BLT,Old Oak Tap,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Fried Bologna,Au Cheval,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Woodland Mushroom,Xoco,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Roast Beef,Al’s Deli,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,PB&L,Publican Quality Meats,https://www.chicagomag.com/Chicago-Magazine/No...


In [23]:
df.to_csv('../data/data03/03. best_sandwiches_list_chicago.csv', sep=',', encoding='utf-8')

---
## 5. 메뉴 상세 페이지에서 가격, 가게 주소 정보 호출하기
- URL 컬럼에 각 메뉴의 상세 페이지에 대한 링크 주소 값을 저장하였다.
- 이 링크에 접속하여 가격, 가게 주소 정보를 가져와보자.
- 가격, 가게 주소 문자열이 담긴 태그는 addy 클래스의 p 태그

In [26]:
# requirements
import re
import pandas as pd
from tqdm import tqdm
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup as bs

df = pd.read_csv('../data/data03/03. best_sandwiches_list_chicago.csv', index_col=0)

df.head()

,Rank,Menu,Cafe,URL
0,1,BLT,Old Oak Tap,https://www.chicagomag.com/Chicago-Magazine/No...
1,2,Fried Bologna,Au Cheval,https://www.chicagomag.com/Chicago-Magazine/No...
2,3,Woodland Mushroom,Xoco,https://www.chicagomag.com/Chicago-Magazine/No...
3,4,Roast Beef,Al’s Deli,https://www.chicagomag.com/Chicago-Magazine/No...
4,5,PB&L,Publican Quality Meats,https://www.chicagomag.com/Chicago-Magazine/No...


- 첫 번째 메뉴의 상세 페이지 URL 값

In [27]:
df['URL'][0]

'https://www.chicagomag.com/Chicago-Magazine/November-2012/Best-Sandwiches-in-Chicago-Old-Oak-Tap-BLT/'

- addy 클래스를 가진 p 태그 호출

In [30]:
req = Request(df['URL'][0], headers={'user-agent':'Chrome'})
res = urlopen(req)
soup = bs(res, "html.parser")

soup.select_one(".addy").get_text()

'\n$10. 2109 W. Chicago Ave., 773-772-0406, theoldoaktap.com'

- 가격($10.)과 가게 주소(2109 W. Chicago Ave.)가 하나의 문자열로 되어있다.
- 이를 해결하기 위해 정규표현식 사용

In [39]:
tmp_string = soup.select_one(".addy").get_text()

re.split(",", tmp_string)

['\n$10. 2109 W. Chicago Ave.', ' 773-772-0406', ' theoldoaktap.com']

In [43]:
tmp = re.split(",", tmp_string)[0]

price_tmp = re.search("\$\d+\.(\d+)?", tmp).group()

price_tmp

'$10.'

In [46]:
address_tmp = tmp[len(price_tmp) + 2:]

address_tmp

'2109 W. Chicago Ave.'

---
# 6. 전체 상세 페이지에서 가격, 주소 정보 호출하기
- 위에서 1위 메뉴의 상세 페이지에서 가격, 주소 정보를 호출할 수 있었다.
- 반복문을 통해 50개 모든 메뉴의 상세 페이지에서 각각의 가격과 주소 정보를 가져와보자.

In [53]:
price = []
address = []

for n in tqdm(df.index) :
    req = Request(df["URL"][n], headers={"user-agent":"Chrome"})
    res = urlopen(req)
    soup = bs(res, "html.parser")
    tmp_string = soup.select_one(".addy").get_text()

    tmp = re.split(",", tmp_string)[0]                  # 가격, 주소 정보
    price_tmp = re.search("\$\d+\.(\d+)?", tmp).group() # 가격만 추출
    address_tmp = tmp[len(price_tmp) + 2:]        # 주소만 추출

    price.append(price_tmp)
    address.append(address_tmp)

100%|██████████| 50/50 [01:41<00:00,  2.03s/it]


In [50]:
len(price), len(address)

(50, 50)

In [51]:
price[:5]

['$10.', '$9.', '$9.50', '$9.40', '$10.']

In [54]:
address[:5]

['2109 W. Chicago Ave.',
 '800 W. Randolph St.',
 ' 445 N. Clark St.',
 ' 914 Noyes St.',
 '825 W. Fulton Mkt.']

- 가격, 주소 리스트를 데이터프레임에 추가해보자
- URL에서 가져올 수 있는 정보는 모두 가져왔으니 URL 컬럼은 삭제
- Rank를 데이터프레임의 인덱스로 변경

In [59]:
df['Price'] = price
df['Address'] = address

final_df = df.drop('URL', axis=1)

final_df.set_index('Rank', inplace=True)

final_df.head()

,Menu,Cafe,Price,Address
Rank,,,,
1,BLT,Old Oak Tap,$10.,2109 W. Chicago Ave.
2,Fried Bologna,Au Cheval,$9.,800 W. Randolph St.
3,Woodland Mushroom,Xoco,$9.50,445 N. Clark St.
4,Roast Beef,Al’s Deli,$9.40,914 Noyes St.
5,PB&L,Publican Quality Meats,$10.,825 W. Fulton Mkt.


In [61]:
final_df.to_csv('../data/data03/03. best_sandwiches_list_chicago2.csv', sep=',', encoding='utf-8')

---
## 7. 주소값을 통한 지도 시각화

- google map api 호출

In [60]:
import folium
import googlemaps
import numpy as np

gmaps_key = "<gmaps api key>"
gmaps = googlemaps.Client(key=gmaps_key)

- 1위 메뉴를 판매하는 식당 주소를 통해 위치 정보 호출해보기

In [80]:
target_name = final_df.iloc[0, 3] + "," + "Chicago"
gmaps_output = gmaps.geocode(target_name)

gmaps_output

[{'address_components': [{'long_name': '2109',
    'short_name': '2109',
    'types': ['street_number']},
   {'long_name': 'West Chicago Avenue',
    'short_name': 'W Chicago Ave',
    'types': ['route']},
   {'long_name': 'West Town',
    'short_name': 'West Town',
    'types': ['neighborhood', 'political']},
   {'long_name': 'Chicago',
    'short_name': 'Chicago',
    'types': ['locality', 'political']},
   {'long_name': 'Cook County',
    'short_name': 'Cook County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Illinois',
    'short_name': 'IL',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '60622', 'short_name': '60622', 'types': ['postal_code']},
   {'long_name': '4821',
    'short_name': '4821',
    'types': ['postal_code_suffix']}],
  'formatted_address': '2109 W Chicago Ave, Chicago, IL 60622, USA',
  'geometry': {'bou

In [77]:
location_output = gmaps_output[0]["geometry"]

location_output

{'bounds': {'northeast': {'lat': 41.8957463, 'lng': -87.6798563},
  'southwest': {'lat': 41.8954846, 'lng': -87.6800603}},
 'location': {'lat': 41.8955886, 'lng': -87.6799623},
 'location_type': 'ROOFTOP',
 'viewport': {'northeast': {'lat': 41.8970239802915,
   'lng': -87.67860931970849},
  'southwest': {'lat': 41.8943260197085, 'lng': -87.6813072802915}}}

In [83]:
lat_tmp = location_output["location"]["lat"]
lng_tmp = location_output["location"]["lng"]

lat_tmp, lng_tmp

(41.9431632, -87.6445071)

- 전체 데이터프레임 내의 Address 값을 통해 모든 식당의 위도, 경도 정보 가져오기

In [82]:
lat = []
lng = []

for idx, row in tqdm(df.iterrows()) :
    # Address 값이 "Multiple location"이면 nan으로 처리
    if not row["Address"] == "Multiple location" :
        target_name = row["Address"] + "," + "Chicago"
        gmaps_output = gmaps.geocode(target_name)
        location_output = gmaps_output[0].get("geometry")

        # 위도
        lat.append(location_output["location"]["lat"])
        # 경도
        lng.append(location_output["location"]["lng"])

    else :
        lat.append(np.nan)
        lng.append(np.nan)

50it [00:06,  7.64it/s]


In [84]:
len(lat), len(lng)

(50, 50)

- 위도, 경도 값을 데이터프레임에 추가

In [85]:
final_df['lat'] = lat
final_df['lng'] = lng

final_df.head()

,Menu,Cafe,Price,Address,lat,lng
Rank,,,,,,
1,BLT,Old Oak Tap,$10.,2109 W. Chicago Ave.,41.895589,-87.679962
2,Fried Bologna,Au Cheval,$9.,800 W. Randolph St.,41.884639,-87.647590
3,Woodland Mushroom,Xoco,$9.50,445 N. Clark St.,41.890523,-87.630783
4,Roast Beef,Al’s Deli,$9.40,914 Noyes St.,42.058322,-87.683748
5,PB&L,Publican Quality Meats,$10.,825 W. Fulton Mkt.,41.886604,-87.648536


- folium을 활용하여 지도에 각 식당 위치를 시각화
- 시카고 위도, 경도 : 41.8781136, -87.6297982

In [93]:
cafe_map = folium.Map(location=[41.8781136, -87.6297982], zoom_start=11)

for idx, row in tqdm(final_df.iterrows()) :
    if not row["Address"] == "Multiple location" :
        folium.Marker(
            location=[row['lat'], row['lng']],
            popup=row['Cafe'],
            tooltip=row['Menu'],
            icon=folium.Icon(
                icon="coffee",
                prefix='fa'
            )
        ).add_to(cafe_map)

cafe_map

50it [00:00, 3477.81it/s]
